In [1]:
import numpy as np
import pandas as pd
from utils import read_q_mat, load_a_mat
import scipy.io


In [2]:
yenbai = read_q_mat('/Users/cooper/Desktop/optimal-wat-allocation/Script/MAT_data/q_yenbai.mat')
vuquang = read_q_mat('/Users/cooper/Desktop/optimal-wat-allocation/Script/MAT_data/q_vuquang.mat')
hoabinh = load_a_mat("/Users/cooper/Desktop/optimal-wat-allocation/Script/MAT_data/a_hoabinh_K.mat")

In [3]:
hoabinh

,Streamflow_[m3/s]
Time,
1956-01-01,548.0
1956-01-02,540.0
1956-01-03,532.0
1956-01-04,520.0
1956-01-05,513.0
...,...
2008-12-27,700.0
2008-12-28,671.0
2008-12-29,727.0


In [4]:
df = load_a_mat(
    "/Users/cooper/Desktop/optimal-wat-allocation/Script/MAT_data/a_hoabinh_K.mat"
)

In [5]:
df

,Streamflow_[m3/s]
Time,
1956-01-01,548.0
1956-01-02,540.0
1956-01-03,532.0
1956-01-04,520.0
1956-01-05,513.0
...,...
2008-12-27,700.0
2008-12-28,671.0
2008-12-29,727.0
